# Random Forest Model

RF model to be used for predicting protein coding genes in DNA sequences.

**Random Search with CV**
- bootstrap': False
- max_depth': 20
- max_features': 'auto'
- min_samples_leaf': 1
- min_samples_split': 2
- n_estimators': 1000
 
**Grid Search with CV**
- bootstrap': False
- max_depth': 40
- max_features': 3
- min_samples_leaf': 1
- min_samples_split': 2
- n_estimators': 2000

BEST MODEL G1 (high default score):
- Random Search with CV - BUT
- n_estimators': 200
- n_features: 200

BEST MODEL G2 (low default score):
- Random search with CV
- n_estimators': 1000
- max_features': 'auto'
- max_depth': 10
- bootstrap': False

## Import packages and choose data

In [ ]:
# Preprocessing and encoding variables
import pandas as pd
import numpy as np
from random import sample
from time import time
import sklearn

# Using Skicit-learn to split data into training and testing sets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Visualising feature importance and making plots
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# Confusion matrix
from sklearn.metrics import plot_confusion_matrix, precision_score, recall_score, roc_auc_score

# Selecting best features
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel

In [ ]:
sns. __version__

### Combining 5 genomes

In [ ]:
G1 = pd.read_csv('../input/genomes-part1/G1.features.csv').iloc[:, 1:]
G2 = pd.read_csv('../input/genomes-part1/G2.features.csv').iloc[:, 1:]
G3 = pd.read_csv('../input/genomes-part1/G3.features.csv').iloc[:, 1:]
G4 = pd.read_csv('../input/genomes-part1/G4.features.csv').iloc[:, 1:]
G5 = pd.read_csv('../input/genomes-part1/G5.features.csv').iloc[:, 1:]

In [ ]:
G1 = G1.sample(2000)
G2 = G2.sample(2000)
G3 = G3.sample(2000)
G4 = G4.sample(2000)
G5 = G5.sample(2000)

In [ ]:
frames = [G1, G2, G3, G4, G5]
df = pd.concat(frames)
df.tail(10)

In [ ]:
G = df

## Single genome

In [ ]:
G = pd.read_csv('../input/genomespart2/G15.features.csv').iloc[:, 1:]

In [ ]:
G.head(10)

## Pre-processing / Data preparation

1. One-hot encoded categorical variables
2. Split data into features and labels
3. Convert to arrays
4. Split data into training and testing sets

In [ ]:
print('The shape of our dataframe is:', G.shape)

In [ ]:
G = G.dropna()

In [ ]:
print('The shape of our dataframe is:', G.shape)

### Encoding target values

In [ ]:
def encode_feature(array):
    """ Encode a categorical array into a number array
    
    :param array: array to be encoded
    :return: numerical array
    """
  
    encoder = preprocessing.LabelEncoder()
    encoder.fit(array)
    return encoder.transform(array)

In [ ]:
class_names = ['CDS', 'LORF']
targets = G["Type"].values
print(targets)

In [ ]:
targets = encode_feature(targets)
print(targets)

In [ ]:
print('The shape of our dataframe is:', G.shape)
print('Rows:', G.shape[0])
print('Columns:', G.shape[1])

### Selecting features and targets and converting data to arrays

In [ ]:
# Labels are the values we want to predict
labels = targets

# Remove the labels from the features -> axis 1 refers to the columns
features = G.drop(['Type','Genome', 'Dataset'], axis = 1)

# Saving feature names as list for later use
feature_names = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [ ]:
print('The shape of our features are:', features.shape)

### Split into training and testing sets

In [ ]:
# Split the data into training and testing sets -> x = features and y = labels/targets
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [ ]:
print('Training Features Shape:', train_x.shape)
print('Training Labels Shape:', train_y.shape)
print('Testing Features Shape:', test_x.shape)
print('Testing Labels Shape:', test_y.shape)

## Train model - default Rand Forest

**RandomForestClassifier**(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
# Create a Gaussian Classifier
clf = RandomForestClassifier(random_state = 42, n_jobs=-1)

# Train the model using the training sets
clf.fit(train_x, train_y)

### Make predictions on test set

In [ ]:
pred_y = clf.predict(test_x)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_y, pred_y))

## Hyperparametertuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 70, num = 10)]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# Random search of parameters, using 3 fold cross validation
clf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 10,
                               cv = 3, verbose=2, random_state=42, n_jobs = -1, return_train_score=True)

# Fit the random search model
clf_random.fit(train_x, train_y);

In [ ]:
clf_random.best_params_

### Make predictions on test set

In [ ]:
pred_y = clf_random.predict(test_x)

# Model Accuracy, how often is the classifier correct?
print("Accuracy after random search:",metrics.accuracy_score(test_y, pred_y))

### Evaluate functions

In [ ]:
def evaluate(model, test_features, test_lables):
    predictions = model.predict(test_features)
    accuracy = metrics.accuracy_score(test_lables, predictions)
    print('Model Performance')
    print('Accuracy = {:0.4f}%.'.format(accuracy))
    return accuracy

#### Default model

In [ ]:
base_accuracy = evaluate(clf, test_x, test_y)

#### Best random search model

In [ ]:
best_random = clf_random.best_estimator_
random_accuracy = evaluate(best_random, test_x, test_y)

In [ ]:
print('Improvement of {:0.3f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

## Train model - best params RandomSearch

In [ ]:
# Create a Gaussian Classifier
clf_rand = RandomForestClassifier(n_estimators=200, max_features=3, max_depth=10, bootstrap=False, random_state = 42)

# Train the model using the training sets
clf_rand.fit(train_x, train_y)

## Make predictions on test set

In [ ]:
pred_y = clf_rand.predict(test_x)

# Model Accuracy, how often is the classifier correct?
print("Accuracy score:", metrics.accuracy_score(test_y, pred_y))

# Precision, Recall and Roc_AUC score
print("Precision score:", metrics.precision_score(test_y, pred_y))
print("Recall score:", metrics.recall_score(test_y, pred_y))
print("ROC_AUC score:", metrics.roc_auc_score(test_y, pred_y))

## Confusion matrix

In [ ]:
plot_confusion_matrix(clf_rand, test_x, test_y)  
plt.show()

#plt.savefig('confmatrix_g1_uneven.png', dpi=300, bbox_inches='tight', transparent=True)

## Finding important features

1. Create a random forests model.
2. Use the feature importance variable to see feature importance scores.
3. Visualize these scores using the seaborn library.

In [ ]:
feature_imp = pd.Series(clf_rand.feature_importances_, index = feature_names).sort_values(ascending=False)

In [ ]:
features = feature_imp.to_frame()
features.columns = ['Feature importance']

In [ ]:
features

In [ ]:
# Grouping all k-mers
dimers = features[features.index.map(lambda x: "2_mer_" in x)]
trimers = features[features.index.map(lambda x: "3_mer_" in x)]
tetramers = features[features.index.map(lambda x: "4_mer_" in x)]
pentamers = features[features.index.map(lambda x: "5_mer_" in x)] 
hexamers = features[features.index.map(lambda x: "6_mer_" in x)]

# Grouping all aa-mers
single_aa = features[features.index.map(lambda x: "1_aa_mer_" in x)]
double_aa = features[features.index.map(lambda x: "2_aa_mer_" in x)]
triple_aa = features[features.index.map(lambda x: "3_aa_mer_" in x)]

# Grouping c_weights
c_weight = features[features.index.map(lambda x: "c_weight" in x)]

#### Mean feature importances of all k-mers, aa-mers and c_weights

In [ ]:
data = {'Feature importance': [round(dimers.mean().iloc[0], 6), round(trimers.mean().iloc[0], 6), 
                               round(tetramers.mean().iloc[0], 6), round(pentamers.mean().iloc[0], 6), 
                               round(hexamers.mean().iloc[0], 6), round(single_aa.mean().iloc[0], 6), 
                               round(double_aa.mean().iloc[0], 6), round(triple_aa.mean().iloc[0], 6),
                               round(c_weight.mean().iloc[0], 6)]}

features_cond = pd.DataFrame(data, index = ['dimers', 'trimers', 'tetramers', 'pentamers', 'hexamers', 
                                            'single_aa', 'double_aa', 'triple_aa', 'c_weight'])

#### Sum of all feature importances of k-mers, aa-mers and c_weights

In [ ]:
data = {'Feature importance': [round(dimers.sum().iloc[0], 6), round(trimers.sum().iloc[0], 6), 
                               round(tetramers.sum().iloc[0], 6), round(pentamers.sum().iloc[0], 6), 
                               round(hexamers.sum().iloc[0], 6), round(single_aa.sum().iloc[0], 6), 
                               round(double_aa.sum().iloc[0], 6), round(triple_aa.sum().iloc[0], 6),
                               round(c_weight.sum().iloc[0], 6)]}

features_cond2 = pd.DataFrame(data, index = ['dimers', 'trimers', 'tetramers', 'pentamers', 'hexamers', 
                                            'single_aa', 'double_aa', 'triple_aa', 'c_weight'])

In [ ]:
features_cond

In [ ]:
ind_list = ['GC_content', 'GC1_content', 'GC2_content', 'GC3_content', 
            'Start_ATG', 'Start_GTG', 'Start_TTG', 'Length']

features_condensed = features.loc[ind_list]

In [ ]:
features_condensed

In [ ]:
features_red = pd.concat([features_condensed, features_cond])
features_red = features_red.squeeze()

In [ ]:
features_red = features_red.sort_values(ascending=False)
features_red

In [ ]:
features_red2 = pd.concat([features_condensed, features_cond2])
features_red2 = features_red2.squeeze()

In [ ]:
features_red2 = features_red2.sort_values(ascending=False)
type(features_red2)

In [ ]:
%matplotlib inline

# Creating a bar plot
sns.barplot(x = features_red, y = features_red.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features (mean)")
#plt.savefig('feat_imp_g15_1.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
%matplotlib inline

# Creating a bar plot
sns.barplot(x = features_red2, y = features_red2.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features (sum)")
#plt.savefig('feat_imp_g15_2.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
%matplotlib inline

features_condensed = features_condensed.squeeze().sort_values(ascending=False)

# Creating a bar plot
sns.barplot(x = features_condensed, y = features_condensed.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.show()

### Visualization

In [ ]:
feature_imp = pd.Series(clf_rand.feature_importances_, index = feature_names).sort_values(ascending=False)

In [ ]:
selected = feature_imp.iloc[0:15]

In [ ]:
%matplotlib inline

# Creating a bar plot
sns.barplot(x = selected, y = selected.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
#plt.savefig('feat_imp_g15_3.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

## Generating the model on selected features

After removing the least important features the accuracy may increase. This is because one removes misleading data and noise, resulting in increased accuracy. A lesser amount of features also reduces the training time.

In [ ]:
feature_imp = pd.Series(clf_rand.feature_importances_, index = feature_names).sort_values(ascending=False)

In [ ]:
selected = feature_imp.iloc[0:200]
selected_feat = list(selected.index)
important_indices = [feature_names.index(x) for x in selected_feat]

In [ ]:
# Create a Gaussian Classifier
clf_imp = RandomForestClassifier(n_estimators=200, max_features='auto', 
                             max_depth=10, bootstrap=False, random_state = 42)

# Select most important features
important_indices = [feature_names.index(x) for x in selected_feat]

train_important = train_x[:, important_indices]
test_important = test_x[:, important_indices]

In [ ]:
# Train the random forest
clf_imp.fit(train_important, train_y)

In [ ]:
# Make predictions and determine the error
pred_y = clf_imp.predict(test_important)

# Model Accuracy, how often is the classifier correct?
print("Accuracy score:", metrics.accuracy_score(test_y, pred_y))
# Precision, Recall and Roc_AUC score
print("Precision score:", metrics.precision_score(test_y, pred_y))
print("Recall score:", metrics.recall_score(test_y, pred_y))
print("ROC_AUC score:", metrics.roc_auc_score(test_y, pred_y))

#### Confusion matrix

In [ ]:
plot_confusion_matrix(clf_imp, test_important, test_y)  
plt.show()

#plt.savefig('confmatrix_g1_bestmod_uneven.png', dpi=300, bbox_inches='tight', transparent=True)